# 1. Data Exploration & Visualization (GSM8K)

**Objective:** Load the raw GSM8K dataset, parse it into our `(Prompt, CoT, Solution)` format, and visualize its properties. This will help us understand the data before we build any models.

In [1]:
%pip install datasets transformers matplotlib seaborn pandas

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/irene-30/DLAI.git "/content/drive/My Drive/DLAI/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into '/content/drive/My Drive/DLAI'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 125 (delta 30), reused 115 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 36.09 KiB | 947.00 KiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
import sys
import os

# Add the 'src' directory to the Python path
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
%cd /content/DLAI

# Now we can import from our source files
from src.utils import parse_gsm8k_sample
from src.utils import get_llm_tokenizer

## 1.1 Load Raw Data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("gsm8k", "main")
train_data = dataset['train']
test_data = dataset['test']

print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

## 1.2 Inspect a Single Sample

In [ ]:
sample = train_data[0]
print("--- RAW QUESTION ---")
print(sample['question'])

print("\n--- RAW ANSWER (CoT + Solution) ---")
print(sample['answer'])

## 1.3 Parse the Sample

Let's test our utility function `parse_gsm8k_sample` from `src/utils.py`.

In [ ]:
parsed = parse_gsm8k_sample(sample)

if parsed:
    prompt, cot, solution = parsed
    print("--- PARSED PROMPT (P) ---")
    print(f"{prompt!r}")

    print("\n--- PARSED CoT (C) ---")
    print(f"{cot!r}")

    print("\n--- PARSED SOLUTION (S) ---")
    print(f"{solution!r}")
else:
    print("Failed to parse sample.")

## 1.4 Visualize Token Lengths

This is the most important visualization. It will show us how long the **Chain-of-Thought (C)** sequences are. If they are long, it justifies our paper's approach of compressing them.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

# Load our tokenizer to get accurate lengths
tokenizer = get_llm_tokenizer()

lengths = []
for sample in train_data:
    parsed = parse_gsm8k_sample(sample)
    if parsed:
        prompt, cot, solution = parsed
        lengths.append({
            'prompt_len': len(tokenizer.encode(prompt)),
            'cot_len': len(tokenizer.encode(cot)),
            'solution_len': len(tokenizer.encode(solution)),
            'full_text_len': len(tokenizer.encode(prompt + cot + solution))
        })

df_lengths = pd.DataFrame(lengths)
df_lengths.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

sns.histplot(df_lengths['cot_len'], bins=50, ax=ax1, kde=True)
ax1.set_title('Distribution of Chain-of-Thought (CoT) Token Lengths')
ax1.set_xlabel('Token Length')

sns.histplot(df_lengths['full_text_len'], bins=50, ax=ax2, kde=True)
ax2.set_title('Distribution of Full Sample (P+C+S) Token Lengths')
ax2.set_xlabel('Token Length')

plt.show()

print("--- Analysis ---")
print(f"Average CoT length: {df_lengths['cot_len'].mean():.2f} tokens")
print(f"Max CoT length: {df_lengths['cot_len'].max()} tokens")
print("Conclusion: The CoT sequences are often long, making them a good target for compression.")